In [1]:
'''
설문 조사에서 
- 복용약 여부 ( Y/N)
- 복용약 종류 (one-hot)
- 영양제 여부 (Y/N)
- 영양제 종류 (one-hot)
- 질환 여부 (Y/N)
- 질환 종류 (one-hot)
- 가족력 (Y/N)
- 가족력 질환 (one-hot)
- 건강문제 (ont-hot)
- 개선목표 (one-hot)
- 수면시간 (one-hot)
- 운동빈도 (one-hot)
- 식사횟수 (one-hot)
- 채소과일빈도 (one-hot)
- 물섭취 (one-hot)
- 음주 (one-hot)
- 흡연 (one-hot) 
'''


'\n설문 조사에서 \n- 복용약 여부 ( Y/N)\n- 복용약 종류 (one-hot)\n- 영양제 여부 (Y/N)\n- 영양제 종류 (one-hot)\n- 질환 여부 (Y/N)\n- 질환 종류 (one-hot)\n- 가족력 (Y/N)\n- 가족력 질환 (one-hot)\n- 건강문제 (ont-hot)\n- 개선목표 (one-hot)\n- 수면시간 (one-hot)\n- 운동빈도 (one-hot)\n- 식사횟수 (one-hot)\n- 채소과일빈도 (one-hot)\n- 물섭취 (one-hot)\n- 음주 (one-hot)\n- 흡연 (one-hot) \n'

In [2]:
import pandas as pd
from sqlalchemy import create_engine
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, Concatenate
from tensorflow.keras.models import Model

In [3]:
pip install sqlalchemy pymysql pandas

Note: you may need to restart the kernel to use updated packages.


In [47]:
from sqlalchemy import create_engine, text

# 1) 커넥션 설정 (Spring Boot 설정과 동일)
ENGINE = create_engine(
    "mysql+pymysql://admin:myNutrition"
    "@mynutritiondb.czccwgs04ahh.ap-northeast-2.rds.amazonaws.com"
    ":3306/muNutrition_db?charset=utf8mb4"
)

In [15]:
import pandas as pd
from sqlalchemy import create_engine, text

# 1) 커넥션 설정 (Spring Boot 설정과 동일)
ENGINE = create_engine(
    "mysql+pymysql://admin:myNutrition"
    "@mynutritiondb.czccwgs04ahh.ap-northeast-2.rds.amazonaws.com"
    ":3306/muNutrition_db?charset=utf8mb4"
)

# 2) survey 메인 + 매핑 테이블 LEFT JOIN (SQL)
sql = """
SELECT
  s.id AS survey_id,
  s.taking_medication,
  s.taking_supplements,
  s.has_diagnosed_disease,
  s.has_family_history,
  s.has_allergy,
  s.gender,
  s.sleep_time,
  s.exercise_frequency,
  s.meal_count,
  s.vegetable_fruit_intake,
  s.water_intake,
  s.drinking,
  s.smoking,

  -- 일대다 관계에서 ARRAY_AGG 로 묶기 (MySQL 8 이상)
  GROUP_CONCAT(DISTINCT sm.medication)       AS med_types,
  GROUP_CONCAT(DISTINCT ss.supplement)       AS supp_types,
  GROUP_CONCAT(DISTINCT sd.disease)          AS disease_types,
  GROUP_CONCAT(DISTINCT sfh.disease)  AS family_diseases,
  GROUP_CONCAT(DISTINCT sal.allergy)         AS allergy_types,
  GROUP_CONCAT(DISTINCT sc.concern)          AS concerns,
  GROUP_CONCAT(DISTINCT shg.goal)            AS health_goals

FROM survey s
LEFT JOIN survey_medication sm ON sm.survey_id = s.id
LEFT JOIN survey_supplement ss ON ss.survey_id = s.id
LEFT JOIN survey_disease    sd ON sd.survey_id = s.id
LEFT JOIN survey_family_history sfh ON sfh.survey_id = s.id
LEFT JOIN survey_allergy    sal ON sal.survey_id = s.id
LEFT JOIN survey_concern    sc  ON sc.survey_id = s.id
LEFT JOIN survey_goal shg ON shg.survey_id = s.id

WHERE s.completed = TRUE
GROUP BY s.id
"""

df_survey = pd.read_sql_query(text(sql), ENGINE)

print(df_survey)

   survey_id taking_medication taking_supplements has_diagnosed_disease  \
0          2           b'\x00'            b'\x01'               b'\x00'   

  has_family_history has_allergy  gender sleep_time exercise_frequency  \
0            b'\x00'     b'\x00'  FEMALE                     LIGHT_1_3   

   meal_count  ... water_intake        drinking     smoking      med_types  \
0           2  ...     UNDER_1L  ONE_TWO_WEEKLY  NON_SMOKER  ANTICOAGULANT   

  supp_types disease_types family_diseases allergy_types concerns health_goals  
0       None          None            None          None     None         None  

[1 rows x 21 columns]


In [17]:
bool_cols = [
    'taking_medication',
    'taking_supplements',
    'has_diagnosed_disease',
    'has_family_history',
    'has_allergy'
]

# ENUM 컬럼(문자열) 목록
enum_cols = [
    'gender',
    'sleep_time',
    'exercise_frequency',
    'vegetable_fruit_intake',
    'water_intake',
    'drinking',
    'smoking'
]
# One-hot 인코딩
df_enum = pd.get_dummies(df_survey[enum_cols], prefix_sep='=')

In [18]:
multi_cols = [
    'med_types',        # GROUP_CONCAT AS med_types
    'supp_types',
    'disease_types',
    'family_diseases',
    'allergy_types',
    'concerns',
    'health_goals'
]

# 1) 문자열 → 리스트
for c in multi_cols:
    df_survey[c] = df_survey[c].fillna('').str.split(',')

# 2) explode → get_dummies → groupby.max
df_ex = df_survey.copy()
for c in multi_cols:
    df_ex = df_ex.explode(c)

df_multi = pd.get_dummies(df_ex, columns=multi_cols, prefix_sep='=')
# survey_id 별로 합치기
df_multi = df_multi.groupby('survey_id').max().drop(columns=bool_cols+enum_cols, errors='ignore')

In [19]:
# 인덱스를 survey_id 로
df_bool = df_survey.set_index('survey_id')[bool_cols]
df_wide  = pd.concat([df_bool, df_enum.set_index(df_survey.index), df_multi], axis=1).fillna(0)

wide_data = df_wide.values
wide_dim  = wide_data.shape[1]
print("▶ Wide dimension:", wide_dim)

▶ Wide dimension: 20


### Deep - 영양성분

In [77]:
import pandas as pd
from sqlalchemy import create_engine
#1) 커넥션 설정 (Spring Boot 설정과 동일)
ENGINE = create_engine(
    "mysql+pymysql://admin:myNutrition"
    "@mynutritiondb.czccwgs04ahh.ap-northeast-2.rds.amazonaws.com"
    ":3306/muNutrition_db?charset=utf8mb4"
)

df = pd.read_csv('cleaned_nutrition_data.csv', encoding='utf-8')

# 한글 컬럼명을 DB 컬럼명(영문 스네이크케이스)으로 매핑
df = df.rename(columns={
    '식품명':               'food_name',
    '대표식품명':           'rep_food_name',
    '영양성분함량기준량':   'nutrition_standard_amount',
    '식품중량':             'serving_size',
    '에너지(kcal)':         'energy_kcal',
    '단백질(g)':           'protein_g',
    '지방(g)':             'fat_g',
    '탄수화물(g)':         'carbohydrate_g',
    '당류(g)':             'sugar_g',
    '식이섬유(g)':         'dietary_fiber_g',
    '칼슘(mg)':            'calcium_mg',
    '철(mg)':              'iron_mg',
    '나트륨(mg)':          'sodium_mg',
    '비타민 A(μg RAE)':    'vitamin_a_rae_ug',
    '비타민 B(mg RDA)':    'vitamin_b_rda_mg',
    '비타민 C(mg)':        'vitamin_c_mg',
    '비타민 D(μg)':        'vitamin_d_ug',
    '콜레스테롤(mg)':      'cholesterol_mg',
    '포화지방산(g)':       'saturated_fat_g',
    '트랜스지방산(g)':     'trans_fat_g',
})

# nutrition_info 테이블에 삽입
with engine.begin() as conn:
    df.to_sql(
        'food_nutrition',
        con=ENGINE,
        if_exists='append',
        index=False,
        method='multi',
        chunksize=500
    )
print("✅ nutrition_info에 메타 정보 포함 모두 업로드 완료")

✅ nutrition_info에 메타 정보 포함 모두 업로드 완료


In [78]:
df = pd.read_csv('filtered_data.csv', encoding='utf-8')
# 한글 컬럼명을 DB 컬럼명(영문 스네이크케이스)으로 매핑

print("CSV 전체 행 개수:", df.shape[0])

cnt = pd.read_sql_query("SELECT COUNT(*) AS cnt FROM food_nutrition", ENGINE)
print(cnt)

CSV 전체 행 개수: 12485
     cnt
0  14584


In [93]:
# 2) nutrition_info 에 저장된 영양성분 컬럼 리스트 (영어 컬럼명)
deep_cols = [
    'protein_g',        # 단백질(g)
    'fat_g',            # 지방(g)
    'carbohydrate_g',   # 탄수화물(g)
    'sugar_g',          # 당류(g)
    'dietary_fiber_g',  # 식이섬유(g)
    'calcium_mg',       # 칼슘(mg)
    'iron_mg',          # 철(mg)
    'sodium_mg',        # 나트륨(mg)
    'vitamin_a_rae_ug', # 비타민 A(μg RAE)
    'vitamin_b_rda_mg', # 비타민 B(mg RDA)
    'vitamin_c_mg',     # 비타민 C(mg)
    'vitamin_d_ug',     # 비타민 D(μg)
    'cholesterol_mg',   # 콜레스테롤(mg)
    'saturated_fat_g',  # 포화지방산(g)
    'trans_fat_g'       # 트랜스지방산(g)
]

# 3) food_nutrtion 테이블에서 deep_cols 만 읽어오기
df_nutri = pd.read_sql_query(
    "SELECT id, " + ",".join(deep_cols) +
    " FROM food_nutrition",
    ENGINE
).set_index('id')

# 4) 숫자형으로 강제 변환 & 결측치는 0으로 채우기
df_nutri = df_nutri.astype(float).fillna(0.0)

# 5) Min–Max 정규화 (0~1)
scaler   = MinMaxScaler()
deep_data = scaler.fit_transform(df_nutri[deep_cols])
deep_dim  = deep_data.shape[1]

print(f"Deep input dimension: {deep_dim}")  # => 24

Deep input dimension: 15


In [80]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, Concatenate
from tensorflow.keras.models import Model

# 1) 입력 레이어
wide_input = Input(shape=(wide_dim,), name='wide_input')
deep_input = Input(shape=(deep_dim,), name='deep_input')

# 2) Wide 부분: 선형 모델
#    (단순히 그대로 쓰거나 필요 시 Dense 레이어 추가)
wide_out = wide_input

# 3) Deep 부분: 비선형 학습
x = Dense(64, activation='relu')(deep_input)
x = Dropout(0.5)(x)
x = Dense(32, activation='relu')(x)
deep_out = Dense(16, activation='relu')(x)

# 4) 결합
combined = Concatenate()([wide_out, deep_out])

# 5) 최종 출력 (추천 점수 회귀)
score_o = Dense(1, activation='linear', name='score')(combined)

# 6) 모델 생성 & 컴파일
model = Model([wide_input, deep_input], score_o)
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ deep_input          │ (None, 16)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 64)        │      1,088 │ deep_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 64)        │          0 │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 32)        │      2,080 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ wide_input          │ (None, 20)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 16)        │        528 │ dense_7[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 36)        │          0 │ wide_input[0][0], │
│ (Concatenate)       │                   │            │ dense_8[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ score (Dense)       │ (None, 1)         │         37 │ concatenate_2[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,733 (14.58 KB)

 Trainable params: 3,733 (14.58 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# 학습
history = model.fit(
    {'wide_input': wide_data, 'deep_input': deep_data},
    labels,
    batch_size=32,
    epochs=20,
    validation_split=0.2,
    verbose=1
)

### content-based 기반 모델 학습

In [94]:
df_nutri_norm = df_nutri.copy()
df_nutri_norm[deep_cols] = scaler.fit_transform(df_nutri[deep_cols].fillna(0))

In [95]:
print(df_nutri_norm)

       protein_g     fat_g  carbohydrate_g   sugar_g  dietary_fiber_g  \
id                                                                      
1       0.042230  0.019259        0.170879  0.020704         0.032154   
2       0.060811  0.042861        0.036264  0.005394         0.012862   
3       0.059122  0.170721        0.293077  0.074012         0.019293   
4       0.089527  0.144802        0.205385  0.028796         0.057878   
5       0.091047  0.040689        0.285495  0.022450         0.057878   
...          ...       ...             ...       ...              ...   
14580   0.065878  0.016507        0.279890  0.002697         0.032154   
14581   0.053547  0.013322        0.239560  0.002459         0.025723   
14582   0.045608  0.018679        0.234396  0.006901         0.025723   
14583   0.038851  0.005502        0.284725  0.001428         0.022508   
14584   0.015203  0.004199        0.018242  0.002459         0.028939   

       calcium_mg   iron_mg  sodium_mg  vitamin_a_

# 추천알고리즘 메서드

In [83]:
def get_recommendations(def_vec, top_n=5):
    """
    def_vec : 1d array, deep_cols 순서대로 0~1 결핍도
    """
    X = df_nutri_norm[deep_cols].values         # (num_foods, D)
    #  하나: 절대량 보충 점수
    wsum = X.dot(def_vec)                       # (num_foods,)
    # 둘: 코사인 유사도 (비율 보충 점수)
    cos_sim = cosine_similarity(X, def_vec.reshape(1,-1)).flatten()
    # 혼합 점수
    score = wsum * cos_sim

    # top_n 인덱스
    idx = np.argsort(-score)[:top_n]

     # 1) 점수만 있는 중간 DataFrame 생성
    recs = pd.DataFrame({
        'id':           df_nutri.index[idx],
        'score':        score[idx],
        'weighted_sum': wsum[idx],
        'cosine_sim':   cos_sim[idx]
    }).set_index('id')
    
     # 2) 메타(df_meta)와 join
    recs = recs.join(df_meta, how='left')

    # 3) 최종 반환 순서 정리
    return recs[[
        'food_name','serving_size',
        'score','weighted_sum','cosine_sim'
    ]]

In [84]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

np.random.seed(42)
example_def = np.random.rand(len(deep_cols))
print(example_def)

[0.37454012 0.95071431 0.73199394 0.59865848 0.15601864 0.15599452
 0.05808361 0.86617615 0.60111501 0.70807258 0.02058449 0.96990985
 0.83244264 0.21233911 0.18182497 0.18340451]


In [103]:
# 추천 전 준비 단계
df_meta = pd.read_sql_query(
    "SELECT id, food_name, serving_size, energy_kcal, sodium_mg FROM food_nutrition",
    ENGINE
).set_index('id')

print(df_meta)

       food_name serving_size  energy_kcal  sodium_mg
id                                                   
1        토스트(식빵)        230.3         84.0      140.0
2      트위스터 샌드위치        380.6         56.0      124.0
3         파운드케이크        170ml        226.0      119.0
4          햄샌드위치        207ml        185.0      321.0
5            간자장        800ml        155.0      345.0
...          ...          ...          ...        ...
14580   콩나물밥_소고기        440ml        131.0      112.0
14581   콩나물밥_양념장        450ml        110.0      115.0
14582      하이라이스        607.4        111.0       63.0
14583        현미밥        370ml        121.0        1.0
14584     근대 된장국        320ml         12.0      232.0

[14584 rows x 4 columns]


In [85]:
top5 = get_recommendations(example_def, top_n=5)
print("=== 추천 TOP5 ===")
print(top5)

=== 추천 TOP5 ===
      food_name serving_size     score  weighted_sum  cosine_sim
id                                                              
5652       굴비구이        52.5m  2.034477      2.888823    0.704258
11181   김구이_조미김           5g  2.023772      2.724535    0.742796
13955      굴비구이        52.5m  1.633319      2.476985    0.659398
10646      멸치조림         100g  1.268011      2.010898    0.630569
3922      뱅어포튀김        10.2m  1.166006      2.096209    0.556245


### 결핍 시나리오 테스트

In [86]:
# 0) 기존 코드에 정의된 deep_cols, df_nutri_norm, df_meta, get_recommendations() 가 있다고 가정

import pandas as pd
import numpy as np

# 1) 다양한 결핍 시나리오 정의
#    – 실제론 설문·로그에서 계산된 결핍 벡터지만, 여기선 예시로 직접 만들어 봅니다.
scenarios = {
    '단백질 결핍 심함':    np.array([0.2, 0.1, 0.9] + [0.5]*(len(deep_cols)-3)),
    '비타민C 결핍 심함':  np.array([0.5]*deep_cols.index('vitamin_c_mg') + \
                                     [0.9] + \
                                     [0.5]*(len(deep_cols)-deep_cols.index('vitamin_c_mg')-1)),
    '전체 균형 결핍':      np.full(len(deep_cols), 0.6),
    '칼로리 과다(결핍 없음)': np.zeros(len(deep_cols)),
    '섬유질 강조 결핍':    np.array([0.5]*deep_cols.index('dietary_fiber_g') + \
                                     [0.9] + \
                                     [0.5]*(len(deep_cols)-deep_cols.index('dietary_fiber_g')-1))
}

# 2) 각 시나리오별로 Top5 추천 뽑아서 DataFrame으로 정리
results = []
for name, vec in scenarios.items():
    recs = get_recommendations(vec, top_n=5)
    recs = recs.reset_index()  # id를 컬럼으로 올리기
    recs['scenario'] = name
    # 필요한 컬럼만 추려서 results에 추가
    results.append(recs[['scenario','id','food_name','serving_size','score']])

# 3) 모두 합치고 보기
df_results = pd.concat(results, ignore_index=True)
print(df_results)

# 4) (선택) 보기 좋게 피벗 테이블로 정리
pivot = df_results.pivot_table(
    index=['scenario','food_name'], 
    values='score', 
    aggfunc='first'
).reset_index()
print(pivot)

         scenario     id      food_name serving_size     score
0       단백질 결핍 심함  11181        김구이_조미김           5g  1.413463
1       단백질 결핍 심함  13873           미역튀각        15.3m  1.394755
2       단백질 결핍 심함  10667          다시마튀각          50g  1.356739
3       단백질 결핍 심함  10671           미역튀각          50g  1.260754
4       단백질 결핍 심함  10646           멸치조림         100g  1.163926
5      비타민C 결핍 심함  11181        김구이_조미김           5g  1.995466
6      비타민C 결핍 심함  10667          다시마튀각          50g  1.710094
7      비타민C 결핍 심함  10646           멸치조림         100g  1.560006
8      비타민C 결핍 심함  13873           미역튀각        15.3m  1.483151
9      비타민C 결핍 심함   5652           굴비구이        52.5m  1.458918
10       전체 균형 결핍  10646           멸치조림         100g  1.958964
11       전체 균형 결핍  11181        김구이_조미김           5g  1.948133
12       전체 균형 결핍   5652           굴비구이        52.5m  1.869237
13       전체 균형 결핍  13873           미역튀각        15.3m  1.798225
14       전체 균형 결핍  10667          다시마튀각          50g  1

In [105]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# --- 하이퍼파라미터 설정 ---
MAX_CAL      = 500    # 1회 제공 시 최대 권장 칼로리
MAX_SODIUM   = 600    # 1회 제공 시 최대 권장 나트륨 (mg)
ALPHA, BETA  = 0.6, 0.4


def get_recommendations(def_vec: np.ndarray, top_n: int = 5):
    """
    def_vec : 1d array (D,), deep_cols 순서대로 0~1 결핍도
    """
    # 1) 영양 벡터와 메타 가져오기
    X      = df_nutri_norm[deep_cols].values         # (N_foods, D)
    meta   = df_meta[['food_name','serving_size','energy_kcal','sodium_mg']]  # pandas.DataFrame
    
    # 2) 원점수 계산
    wsum    =         X.dot(def_vec)                          # (N_foods,)
    cos_sim = cosine_similarity(X, def_vec.reshape(1,-1)).ravel()  # (N_foods,)
    
    # 3) 0~1 정규화
    wsum_n  = (wsum    - wsum.min()) / (wsum.max()   - wsum.min()   + 1e-9)
    cos_n   = (cos_sim - cos_sim.min()) / (cos_sim.max() - cos_sim.min() + 1e-9)
    
    # 4) 기본 점수 (선형 결합)
    base_score = ALPHA * wsum_n + BETA * cos_n              # (N_foods,)
    
    # 5) 페널티 (칼로리·나트륨 초과 시 0)
    pen_cal   = np.clip(1 - meta['energy_kcal'].values / MAX_CAL, 0, 1)
    pen_sod   = np.clip(1 - meta['sodium_mg'].values   / MAX_SODIUM, 0, 1)
    penalty   = pen_cal * pen_sod                          # (N_foods,)
    
    # 6) 최종 스코어
    final_score = base_score * penalty                      # (N_foods,)
    
    # 7) 상위 top_n 인덱스
    idx = np.argsort(-final_score)[:top_n]
    
    # 8) 결과 DataFrame 구성
    recs = pd.DataFrame({
        'score':        final_score[idx],
        'wsum_norm':    wsum_n[idx],
        'cos_norm':     cos_n[idx],
        'penalty':      penalty[idx],
    }, index=df_nutri_norm.index[idx])
    
    # 9) 메타와 병합
    recs = recs.join(meta, how='left')
    
    # 10) 반환할 컬럼 순서
    return recs[[
        'food_name','serving_size',
        'score','wsum_norm','cos_norm','penalty',
        'energy_kcal','sodium_mg'
    ]]


In [109]:
df_nutri_norm.columns

Index(['protein_g', 'fat_g', 'carbohydrate_g', 'sugar_g', 'dietary_fiber_g',
       'calcium_mg', 'iron_mg', 'sodium_mg', 'vitamin_a_rae_ug',
       'vitamin_b_rda_mg', 'vitamin_c_mg', 'vitamin_d_ug', 'cholesterol_mg',
       'saturated_fat_g', 'trans_fat_g'],
      dtype='object')

In [111]:
import numpy as np

# 1) 현재 deep_cols 순서
deep_cols = [
    'protein_g', 'fat_g', 'carbohydrate_g', 'sugar_g', 'dietary_fiber_g',
    'calcium_mg', 'iron_mg', 'sodium_mg', 'vitamin_a_rae_ug',
    'vitamin_b_rda_mg', 'vitamin_c_mg', 'vitamin_d_ug',
    'cholesterol_mg', 'saturated_fat_g', 'trans_fat_g'
]
D = len(deep_cols)
base_val = 0.5  # 기본 결핍도

# 2) 시나리오별 결핍 벡터 생성
# — 단백질 결핍 심함
vec_p = np.full(D, base_val)
vec_p[deep_cols.index('protein_g')] = 1.0

# — 비타민 C 결핍 심함
vec_c = np.full(D, base_val)
vec_c[deep_cols.index('vitamin_c_mg')] = 1.0

# — 전체 균형 결핍
vec_bal = np.full(D, 0.6)

# — 칼로리 과다 (결핍 없음)
vec_none = np.zeros(D)

# — 섬유질 강조 결핍
vec_f = np.full(D, base_val)
vec_f[deep_cols.index('dietary_fiber_g')] = 1.0

# 3) 딕셔너리로 모아서
scenarios = {
    '단백질 결핍 심함':       vec_p,
    '비타민C 결핍 심함':     vec_c,
    '전체 균형 결핍':         vec_bal,
    '칼로리 과다(결핍 없음)': vec_none,
    '섬유질 강조 결핍':       vec_f
}

# 4) 확인
for name, vec in scenarios.items():
    print(name, vec)

단백질 결핍 심함 [1.  0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
비타민C 결핍 심함 [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 1.  0.5 0.5 0.5 0.5]
전체 균형 결핍 [0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6]
칼로리 과다(결핍 없음) [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
섬유질 강조 결핍 [0.5 0.5 0.5 0.5 1.  0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]


In [112]:
# 0) 기존 코드에 정의된 deep_cols, df_nutri_norm, df_meta, get_recommendations() 가 있다고 가정

import pandas as pd
import numpy as np

# 1) 각 시나리오별로 Top5 추천 뽑아서 DataFrame으로 정리
results = []
for name, vec in scenarios.items():
    recs = get_recommendations(vec, top_n=5)
    recs = recs.reset_index()  # id를 컬럼으로 올리기
    recs['scenario'] = name
    # 필요한 컬럼만 추려서 results에 추가
    results.append(recs[['scenario','id','food_name','serving_size','score']])

# 2) 모두 합치고 보기
df_results = pd.concat(results, ignore_index=True)
print(df_results)

         scenario     id      food_name serving_size     score
0       단백질 결핍 심함    399         돼지고기_간         100g  0.378471
1       단백질 결핍 심함  11098         미역국_새우         400g  0.352636
2       단백질 결핍 심함   5891           불낙전골        276.3  0.348147
3       단백질 결핍 심함   4208           불낙전골        276.3  0.348147
4       단백질 결핍 심함   4033            유산슬        136.3  0.344998
5      비타민C 결핍 심함   3489          돌나물무침        45.1m  0.387736
6      비타민C 결핍 심함   4587           오이무침        108.2  0.373637
7      비타민C 결핍 심함  13804           오이무침        108.2  0.373637
8      비타민C 결핍 심함   4501           딸기넥타        355ml  0.364634
9      비타민C 결핍 심함  14200      딸기바나나 스무디        355ml  0.364527
10       전체 균형 결핍    399         돼지고기_간         100g  0.382063
11       전체 균형 결핍  11098         미역국_새우         400g  0.359136
12       전체 균형 결핍   4499        두유 카페라떼        355ml  0.348265
13       전체 균형 결핍   4477           카페라떼        355ml  0.342021
14       전체 균형 결핍    150     커피_카라멜마끼아토         100g  0